# EE 120 Lab 8: Shazam
v1 - Spring 2020: Anmol Parande, Dominic Carrano

# Background

In 2002, Shazam Entertainment Limited (founded by UC Berkeley students!) launched its music identification product, allowing users to dial a phone number and play a song. Then, they'd get a text message with the name of the song and its artist. In 2018, Shazam was acquired by Apple for \$400 million, and it's now in every iPhone.

Shazam works by using *audio fingerprinting*: given a song, it generates a set of identifiers, and searches an audio database to find a match and identify the song. In this lab, you'll learn about audio fingerprinting, and use it to build a music identification just like Shazam!

## Dependencies

To get started, you'll need to install [Pandas](https://pandas.pydata.org/docs/getting_started/install.html). And, of course, you'll also need NumPy, SciPy, and Matplotlib if you don't already have them.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import pandas as pd
import IPython.display as ipd

from scipy.io import wavfile
from scipy.ndimage import maximum_filter
from shazam_utils import generate_hash

Note: To avoid any copyright issues, we've cropped all provided songs to only contain the first 60 seconds.

# Q1: Spectral Analysis

As we've already seen throughout the course, a signal's constituent frequencies tell us a lot about it. The same is true of audio: to find the salient features of songs to fingerprint, we'll need to look at the song's spectrum (i.e., Fourier Transform). Fortunately, we have the DFT (efficiently implemented via the FFT) to help us do this.

To get started, let's load in *Viva La Vida* by Coldplay.

In [ ]:
fs, coldplay = wavfile.read("VivaLaVida.wav")
print("Audio Shape: {0}, Sampling Rate: {1} Hz".format(coldplay.shape, fs))
coldplay = np.mean(coldplay, axis=1)

Typically, we think of audio as a two-channel, continuous signal $\vec{x}(t) = \left[x_L(t) \ x_R(t)\right]$, with one column of the *audio matrix* per channel. That is, $x_L(t)$ is the left channel's signal, and $x_R(t)$ the right channel's signal. The reason we have two distinct audio channels is so that we can have two streams playing at the same time, one per ear (e.g., in a pair of headphones or laptop speakers).

We sample this CT audio signal at a particular rate (here, 48000 Hz) to get a DT signal. For our purposes, the distinction between our channels is not very important, so we'll just average them to form a 1D signal, $x(n)$.

To get a sense for the song we're working with, feel free to have a listen! This cell may take a few seconds to run.

In [ ]:
ipd.Audio("VivaLaVida.wav")

## Q1a: One DFT is Not Enough

As far as spectral analysis is concerned, it seems like we should just be able to take the DFT of the entire song, find our fingerprints, and be done, right? Is that really all there is to Shazam? No, not quite. It may not be obvious, but there's a big issue with this approach that we'll explore now. So that our code doesn't take forever to run, we'll only look at the first 10 seconds of the song, but the issues we'll find here apply generally to the entire signal.

To see why one DFT won't suffice, in the cell below:
1. Define a new signal `coldplay_cropped` using the first 10 seconds of the signal `coldplay`. **Make sure to use the variable `fs` defined for you.**
2. Compute `coldplay_freqs`, the magnitude of its spectrum; you may use numpy for this (check out [np.fft.fft](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fft.html) and [np.abs](https://docs.scipy.org/doc/numpy/reference/generated/numpy.absolute.html)).
3. Center the magnitude with [np.fft.fftshift](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fftshift.html). By default, when you compute the FFT, the samples of the DTFT that are returned go from $0$ to $2\pi$; fftshift moves them around so that they go from $-\pi$ to $\pi$, which is nicer for visualization.

In [ ]:
# TODO: Compute the magnitude spectrum of the first 10 seconds of Viva La Vida


Let's see what the spectrum of the first 10 seconds of the song looks like.

In [ ]:
plt.figure(figsize=(16, 4), dpi=200)
freqs = np.linspace(-fs/2, fs/2, len(coldplay_freqs))
plt.plot(freqs, coldplay_freqs)
plt.xlabel("Frequency [Hz]")
plt.ylabel("Magnitude")
plt.title("DFT of first 10 seconds of Viva La Vida")
plt.show()

Most of the frequency content is centered around the lower frequencies. In fact, we can barely see anything past 10 kHz, because human hearing stops around 15-20 kHz (and generally decreases with age), so there's no reason to include anything that high in music.

So far, everything looks ok: we got the spectrum of the first 10 seconds of our song. This gives us a sort of "aggregate view" of the frequencies that show up at some point during the first 10 seconds. But is this "aggregate view" good enough? What happens if our signal is *non-stationary*, that is, is frequency content changes with time, as is certainly the case with music? 

To find out, in the cell below, define `coldplay_freqs_1`, `coldplay_freqs_2`, `coldplay_freqs_3`, `coldplay_freqs_4` as the centered (via fftshift) magnitude spectra of the first, second, third, and fourth seconds of the song, respectively.

In [ ]:
# TODO your code here


Let's use these variables to zoom in (temporally speaking) and inspect the song's frequency content over the course of a second of data (rather than 10), and see if the "aggregate view" gives a good enough picture of what frequencies are present at a specific second in time.

In [ ]:
freqs = np.linspace(-fs / 2, fs / 2, len(coldplay_freqs_1))
sigs = [coldplay_freqs_1, coldplay_freqs_2, coldplay_freqs_3, coldplay_freqs_4]
strs = ["1st", "2nd", "3rd", "4th"]

plt.figure(figsize=(16, 10), dpi=200)
for i in range(1, 5):
    plt.subplot(2, 2, i)
    plt.plot(freqs, sigs[i-1])
    plt.xlim([-.5e4, .5e4])
    plt.ylim([0, 1.1 * np.array(sigs).max()])
    plt.title("DFT magnitude of {} second of Viva La Vida".format(strs[i-1]))
plt.show()

Notice how while most of the energy in each second's spectrum is concentrated inside $[-2.5 \text{ kHz}, +2.5 \text{ kHz}]$, the exact shapes are quite different. 

**The issue is that the aggregate view from our 10-second DFT doesn't have good enough *temporal resolution*: we can't see how the signal's frequency content changes over time!**

Why does this matter, you ask? Well, when we're working with the real deal, we don't feed Shazam the entire song; only a clip. For example, suppose you tune into a radio station halfway through a song. Then, 20 seconds later, you think to yourself, "hey, I like this" and pull out Shazam to figure out what song it is. By then, whatever you're giving Shazam is missing a lot of data, and so it needs to be able to look at what frequencies are in the song at different points in time to correctly identify it. The aggregate view won't do. Fortunately, there's a very simple fix to this.

## Q1b: Spectrogrammin'

The results of Q1a are pretty clear: we need a way to see how the signal's frequency content changes over time. Just taking one DFT of the entire signal fails to achieve this. Instead, we'll use a *spectrogram*.

### Spectrograms

A *spectrogram* is an image representing the frequency content of a signal at different times. This ability to see how a signal's frequency content changes with time is the key useful feature of a spectrogram. 

To compute a spectrogram, we split our signal into chunks, compute the DFT of each chunk, and plot the magnitude squared of those DFT chunks side-by-side. To make visualization easier, we typically employ a colormap to distinguish where the DFT's squared-magnitude is bigger.

For example, here is a spectrogram of speech, taken from [here](https://www.researchgate.net/figure/Spectrogram-of-a-speech-signal-with-breath-sound-marked-as-Breath-whose-bounds-are_fig1_319081627). The red areas correspond to stronger frequency content, and green areas to weaker frequency content.

Notice the differences between when the speaker takes a breath and when the speaker is actually speaking. A single DFT wouldn't be able to separate this!

![image.png](https://www.researchgate.net/profile/Sri_Harsha_Dumpala/publication/319081627/figure/fig1/AS:534034566004736@1504335170521/Spectrogram-of-a-speech-signal-with-breath-sound-marked-as-Breath-whose-bounds-are.png)

### Your Job

To get some familiarity with spectrograms, let's generate some sinusoidal signals and plot their spectrograms.

In the cell below, generate 1000 samples of the following signals over the interval $t \in [0, 1]$:
- A 100 Hz sine wave (call it `x1`).
- A 400 Hz sine wave (call it `x2`).

Finally, create a third signal, call it `x3`, by concatenating `x1` and `x2`. 

As a reminder, a sine wave at frequency $f_0$ Hz is given by the equation $x(t) = \sin(2\pi f_0 t)$.

In [ ]:
# TODO: Generate 3 sinusoidal signals


First, let's look at the DFT of our signals.

In [ ]:
freqs_1 = np.abs(np.fft.fftshift(np.fft.fft(x1)))
freqs_2 = np.abs(np.fft.fftshift(np.fft.fft(x2)))
freqs_3 = np.abs(np.fft.fftshift(np.fft.fft(x3)))

fig, axs = plt.subplots(1, 3, figsize=(16, 4), dpi=200)

axs[0].plot(np.linspace(-500,500, len(freqs_1)), freqs_1)
axs[0].set_ylabel('DFT Magnitude')
axs[0].set_xlabel('Frequency [Hz]')
axs[0].set_title("100 Hz Signal")

axs[1].plot(np.linspace(-500,500, len(freqs_2)), freqs_2)
axs[1].set_ylabel('DFT Magnitude')
axs[1].set_xlabel('Frequency [Hz]')
axs[1].set_title("400 Hz Signal")

axs[2].plot(np.linspace(-500,500, len(freqs_3)), freqs_3)
axs[2].set_ylabel('DFT Magnitude')
axs[2].set_xlabel('Frequency [Hz]')
axs[2].set_title("100 Hz and 400 Hz Signal")
plt.show()

As expected, the *pure tones* (the 100 Hz and 400 Hz sine waves) have 2 peaks each, due to conjugate symmetry, whereas the signal formed by concatenating them has 4 peaks. Now, let's look at the spectrograms of these signals. Run the following code to plot the spectrogram of each signal.

In [ ]:
f1, t1, x1_freqs = signal.spectrogram(x1, fs=1000)
f2, t2, x2_freqs = signal.spectrogram(x2, fs=1000)
f3, t3, x3_freqs = signal.spectrogram(x3, fs=1000)

fig, axs = plt.subplots(1, 3, figsize=(16, 4), dpi=200)

axs[0].pcolormesh(t1, f1, x1_freqs, cmap="gray")
axs[0].set_ylabel('Frequency [Hz]')
axs[0].set_xlabel('Time [sec]')
axs[0].set_title("100 Hz Signal")

axs[1].pcolormesh(t2, f2, x2_freqs, cmap="gray")
axs[1].set_ylabel('Frequency [Hz]')
axs[1].set_xlabel('Time [sec]')
axs[1].set_title("400 Hz Signal")

axs[2].pcolormesh(t3, f3, x3_freqs, cmap="gray")
axs[2].set_ylabel('Frequency [Hz]')
axs[2].set_xlabel('Time [sec]')
axs[2].set_title("100 Hz and 400 Hz Signal")

plt.show()

If you did this correctly, you should see 3 spectrograms. The first one will have a single band at 100 Hz. The second will have a single band at 400 Hz. The final one will have two bands (one at 100 Hz and one at 400 Hz). The reason we aren't seeing conjugate symmetry here is because we are only plotting the positive frequencies. For the most part, these spectrograms appear to give us the same information as the DFT. 

However, notice that in the 3rd spectrogram, the frequencies are mostly only present for the duration they exist. There's some overlap between 1.0-1.2 seconds, which isn't what we would have expected. This happens because SciPy doesn't truly use distinct chunks, as we mentioned above, and instead goes with a more sophisticated overlapping window approach, covered in EE 123 (this gives a better tradeoff between the temporal and spectral resolutions).

## Q1c: Spectrograms of Songs

Now that we've got the basic concepts down, let's load *Viva La Vida* and *Mr. Brightside* and compare their spectrograms. Run the cell below to load the songs.

In [ ]:
fs, coldplay = wavfile.read("VivaLaVida.wav")
coldplay = np.mean(coldplay, axis=1)

fs, killers = wavfile.read("MrBrightside.wav")
killers = np.mean(killers, axis=1)

Since we haven't heard *Mr. Brightside* yet, let's load it in now and have a listen. This cell will take a few seconds to load before the audio interface shows up.

In [ ]:
ipd.Audio("MrBrightside.wav")

To get a better looking image when visualizing the spectrogram, we'll plot everything in decibels. 

Recall that to convert a number $x$ to decibels, we compute $x_\text{dB} = 20\log_{10}(x).$

### Your Job

In the cell below:
1. Use [`signal.spectrogram`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html) to compute the spectrogram of each song. 
    - Use 4096 for the `nperseg` parameter of `signal.spectrogram` to take a 4096 point DFT. This matches the length of DFT typically used in practical audio fingerprinting systems, representing a good tradeoff between spectral and temporal resolution.
    - The function returns a tuple containing the frequencies of the spectrogram samples, time points of the spectrogram samples, and the actual spectrogram. For each call to `signal.spectrogram`, store these as `fi, ti, []_spect` where `i` is 1 or 2 and `[]` gets filled with `coldplay` or `killers` depending on the song.
2. Convert the resultant spectrograms to the decibel scale using the formula from above.

Don't worry about any divide-by-zero issues that happen due ot taking the log. If you're concerned, you can add a small positive constant (say, $10^{-12}$) as a remedy to silence the warning.

In [ ]:
# TODO: Compute the spectrogram of Viva La Vida and Mr. Brightside in decibels


Let's have a look! Run the cell below to plot the spectrograms.

In [ ]:
plt.figure(figsize=(20, 10), dpi=200)

plt.subplot(2, 1, 1)
plt.pcolormesh(t1, f1, coldplay_spect, cmap="jet")
plt.ylabel("Frequency [Hz]")
plt.xlabel("Time [sec]")
plt.title("Viva La Vida")
plt.colorbar()

plt.subplot(2, 1, 2)
plt.pcolormesh(t2, f2, killers_spect, cmap="jet")
plt.ylabel("Frequency [Hz]")
plt.xlabel("Time [sec]")
plt.title("Mr. Brightside")
plt.colorbar()

plt.show()

<hr>

**Q:** In both spectrograms, we see a column of dark blue for the first second or so. Based on our colorbar, it looks like this corresponds to $\approx -300 \text{dB}$, or essentially no signal power. In terms of the songs, why do we have this in our plots?

<span style="color:blue">**A:** (TODO)</span>

<hr>


**Q:** At the beginning of the spectrogram for Mr. Brightside (after the column of dark blue), you should see two peaks that extend up toward $20 \text{ kHz}$. What sound in the song is this part of the spectrogram capturing?

<span style="color:blue">**A:** (TODO)</span>

<hr>

**Q:** Can you easily tell the two songs' spectrograms apart? Do you think they'd make good building blocks for our audio recognition algorithm?

<span style="color:blue">**A:** (TODO)</span>

<hr>

# Q2: Fingerprinting

Our end goal here is to take an audio snippet, and figure out what song's being played. To do this, we'll need a database of songs to compare against. 

Should we just store entire songs in the database? Probably not, as that'd be a very large database: a three-minute WAV file sampled at $48 \text{ kHz}$ is a about $30 \text{ MB}$ in size. Even if we aimed for the modest goal of 1000 songs (which the original iPod from 2001 could hold), we're already looking at using over $30 \text{ GB}$ of storage. Additionally, comparing raw audio samples for similarity isn't very robust against noise.

Instead, we'll generate a set of *fingerprints* from each song, and store these in our database. When our version of Shazam gets fed a song to classify, it can just compare the fingerprints, rather than looking at the whole song. This should solve our storage issues, provided the fingerprints aren't too large. But clearly we'll need this fingerprinting algorithm to have a few other properties for this audio recognition system to be useful.

In particular, we want our audio fingerprint to have four key properties:
1. ***Temporal Locality:*** We're trying to figure out what song is being played based on a short (say, 5 to 10 second long) clip. So, our fingerprints should somehow encode *where* in the song they come from.

2. ***Translational Invariance:*** The snippet we play for Shazam could come from anywhere in the song. We could play it the first 5 seconds, the last 5, or something in the middle. In all cases, we want a correct result, so the same chunk of audio should get the same fingerpint regardless of whether it shows up a minute into a clip or right at the beginning—it's the actual music in it that we should use to generate the fingerprint.

3. ***Robustness:*** An audio file, whether clean or degraded by (a modest amount of) noise, should produce the same fingerprint.

4. ***High Entropy:*** The fingerprinting algorithm should be "random enough" that two different songs don't produce the same fingerprint.

As it turns out, spectrograms have all these nice properties, which is why they're such an important part of Shazam! The company's founders recognized this too, and discussed it in their original paper, linked in the references.

**So, spectrograms are cool, but how can we use them? They contain thousands of points... how do we pick which are the most important?**

As you might guess, we'll look at the spectrogram's *peaks*: points in high-energy areas. These are the most likely to survive distortions from noise, unlike ones that are close to zero and easily drowned out.

## Q2a: Peak Finding

To extract these peaks, we want to find areas of the spectrogram where's there's some point that has more energy than its neighbors. To do this, we're going to need some non-linear filtering. 

### Max Filtering 

So far in this course, we have almost entirely worked with LTI systems, as they're amenable to analysis. However, LTI filtering can't help us here because a "maximum" operation is fundamentally non-linear, failing to satisfy the superposition property. 

To do our peak finding, we'll use Scipy's [`maximum_filter`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.maximum_filter.html) function with a neighborhood of 51 (the `size` parameter in the function call). 

For each point in our spectrogram, this filter will take our spectrogram $f(x, y)$ and output $g(x, y)$, the maximum value in a 51x51 region around the pixel. 

Formally,

$$g(x, y) = \max_{i,j} f(x+i, y+j) \text{  where } -25\le i, j \le 25.$$

### Your Job

Implement the maximum filter and apply it to the spectrogram of "Viva La Vida." When the neighborhood exceeds the boundary of the image, assume $f(x, y)$ is the value of the image at that point (i.e., set `mode='constant'`).


After applying the maximum filter:
1. Extract a boolean mask which is True when $f(x, y) = g(x, y)$, and False otherwise.
2. To ensure these peaks are big enough, in the mask, set any peak locations with a peak below `AMP_THRESH` to zero. You can easily, but are not required to, accomplish this with a bitwise `&` operation.
3. Use [`np.nonzero`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.nonzero.html) to convert your mask into a set of (frequency, time) pairs. This function will return two arrays. The first is the indices along the frequency axis of the spectrogram where the peaks show up, and the second is the peak indices along the time axis.

In [ ]:
NEIGHBORHOOD_SIZE = 2 * 25 + 1
AMP_THRESH = 40

# TODO: Apply a Maximum Filter
max_spect = 

# TODO: Compute the mask
mask = 

# TODO: Filter out tiny peaks
mask &= 

# TODO: Get the indices of the peaks
freq_idx, time_idx = 

Let's run the next cell and see where our peaks are! We'll label them with black dots for clarity.

In [ ]:
plt.figure(figsize=(16, 6), dpi=200)
plt.scatter(t1[time_idx], f1[freq_idx], zorder=99, color='k')
plt.pcolormesh(t1, f1, coldplay_spect, zorder=0, cmap="jet")
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title("Spectrogram Peaks (for Viva La Vida)")
plt.xlim([0, 60])
plt.show()

<hr>

**Q:** In Q1, we saw how most of the information in music signals is in the lower frequencies (under, say, $10 \text{ kHz}$). How does this compare with the spectrogram peaks? Are they mostly in lower or upper half of the spectrgram? Is this what you'd expect?

<span style="color:blue">**A:** (TODO)</span>

<hr>

## Q2b: Hashing

The peaks we've found make up what the creators of Shazam call a *constellation map*. We'll use the points in our constellation map to compute the song's fingerprints. 

To do this, we'll take each peak, say $(t_i, f_i)$, and chain it together with the next 15 peaks $(t_{i+1}, f_{i+1}), ..., (t_{i+15}, f_{i+15})$ by:
1. Subtracting the times where the peaks show up, computing $t_d = t_{i+j} - t_i$.
2. Hash the string $[f_i:f_{i+j}:t_d]$. The fingerprint is $(\text{hash}, t_i)$.

Hashing is out of the scope of this course, so we've provided the hashing function for you (`generate_hash`). It takes  arguments $f_i, f_{i+j}, t_i, t_{i+j}$ (in that order), and does steps 1 and 2, returning the fingerprint, $(\text{hash}, t_i)$.

We've provided `sorted_peaks`, a list of all the peaks sorted in increasing order by the time at which they occur in the song. Note that each element of `sorted_peaks` contains two elements, the $(f, t)$ tuple indicating where the peak occured on the spectrogram. That is, `sorted_peaks[i]` contains the tuple $(f_i, t_i)$.

### Your Job

Using this, your job is to, for each peak in `sorted_peaks`:
1. Extract $(f_i, t_i)$.
2. Iterate over the next `HASHES_PER_PEAK` peaks (if you've hit the end of the array, then stop chaining the current peak and move onto the next peak), and:
    - Extract $(f_{i+j}, t_{i+j})$.
    - Generate the hash using the `generate_hash` function with $f_i, f_{i+j}, t_i, t_{i+j}$.
    - Append it to `hashes`.

Remember, we only found the indices of the peaks, not the actual times and frequencies!

In [ ]:
HASHES_PER_PEAK = 15
times, freqs = t1[time_idx], f1[freq_idx]
sorted_peaks = sorted(zip(freqs, times), key=lambda x: x[1])
hashes = []

# TODO populate "hashes" with the fingerprints


Let's look at the first three hashes. If your code's correct, the output of running the next cell should be `[('eb046cc61c9c72cbc3e9', 1.6853333333333333), ('c3732d9a4bc79b82286e', 1.6853333333333333), ('8ff87ca5f63e8a036f46', 1.6853333333333333)]`.

In [ ]:
print(hashes[:5])

Now, we can easily compute the fingerprint of a signal! After fingerprinting, all we need to do is search our database for a match. If we did things correctly, the database entry we have the most fingerprints in common with should match the true song.

Let's move all of this code into a single function so we can easily compute hashes for any audio signal.

In [ ]:
def fingerprint(audio, fs, min_distance=25, amp_thresh=40, hashes_per_peak=15):
    NEIGHBORHOOD_SIZE = 2 * min_distance + 1
    AMP_THRESH = amp_thresh
    HASHES_PER_PEAK = hashes_per_peak
    
    audio = np.mean(audio, axis=1)
    
    # TODO: Compute the spectrogram of the single channel audio (Copy fr) 

    
    # TODO: Apply a Maximum Filter (Copy from Q2a)

    
    # TODO: Compute the mask (Copy from Q2a)

    
    # TODO: Filter out tiny peaks (Copy from Q2a)

    
    # TODO: Get the indices of the peaks (Copy from Q2a)

    
    
    
    times, freqs = t1[time_idx], f1[freq_idx]
    sorted_peaks = sorted(zip(freqs, times), key=lambda x: x[1])
    hashes = []
    # TODO: Compute the hashes (Copy from Q2b)
    
    
    return hashes

# Q3: Testing
As mentioned before, all we need to do now is test our system and make sure it's as robust as we think it is. Our database is stored in `database.csv`. It's columns are |Hash|t1|Song|. A production application with thousands of songs in the database would use SQL or some other querying language, but a simple CSV will suffice for our uses.

Because searching through our database is more of a software problem than a Signals and Systems problem, we've provided the detection function for you. 

This function:
1. Loads the CSV using pandas (a data analysis package),
2. Fingerprints the unknown sample,
3. Searches for matches, and
4. Returns the song with the most matches, its confidence as a percentage.

In [ ]:
def detect(audio, fs):
    db = pd.read_csv("database.csv", header=None, names=["Hash", "time", "Song"])

    hashes = fingerprint(audio, fs)
    db_matches = db[db.Hash.isin(map(lambda x: x[0], hashes))]
    if len(db_matches) == 0:
        print("No Matches")
        return

    counts = db_matches.groupby("Song").size()
    counts = counts / counts.sum()
    return counts.idxmax(), counts.max() * 100

## Q3a: Basic Testing

Let's see how our system does under ideal conditions (i.e, no noise). Take a 20 second segment from Viva La Vida and Mr. Brightside and call the detect function to identify it. We've already reloaded the audio for you.

In [ ]:
fs, coldplay = wavfile.read("VivaLaVida.wav")
fs, killers = wavfile.read("MrBrightside.wav")

In [ ]:
# TODO: Detect a 20 second chunk of Coldplay


In [ ]:
# TODO: Detect a 20 second chunk of Mr. Brightside


## Q3b: Gaussian Noise
We want our system to be robust to different forms of noise. To start with, let's add some Gaussian noise to our audio and try to detect its origin. We'll take a 20 second chunk of Viva La Vida, add Gaussian noise with a mean and variance of 10000, and see if you can identify them. *(Hint: Checkout the [`np.random.normal`](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.normal.html) function)*.

In [ ]:
# TODO add gaussian noise to some 20 sec chunk, call the result unknown_segment_gaussian


Our version of Shazam should still be able to detect the song. How does it sound, though?

In [ ]:
ipd.Audio(unknown_segment_gaussian.T, rate=fs)

It sounds terrible, and we can barely make out the music! Yet, our system still correctly identified it as *Viva La Vida*!

Let's try with *Mr. Brightside*. Now, take a 20 second chunk of the song, add Gaussian noise with a mean and variance of 10000, and see we can still identify them.

In [ ]:
# TODO add gaussian noise to some 20 sec chunk, call the result unknown_segment_gaussian


Again, the song's barely audible, but our identification system still works!

In [ ]:
ipd.Audio(unknown_segment_gaussian.T, rate=fs)

## Q3c: Blocked Speaker

What if instead of Gaussian noise, a portion of the audio just becomes zero? Arguably, this is a more realistic model of how our signal could get corrupted when dealing with music recognition. For example, somebody could move in front of the speaker, pause the music, or turn the volume down very low. 

Take a 20 second chunk of Viva La Vida, zero out five random 2 second chunks, and see if we can still detect the source. Use np.copy if you have copying issues.

In [ ]:
# TODO store in unknown_segment


Let's hear how the song sounds with these portions removed.

In [ ]:
ipd.Audio(unknown_segment.T, rate=fs)

How does Shazam do now? Surely it'll fail with half the clip missing.

In [ ]:
detect(unknown_segment, fs)

What about *Mr. Brightside*?

In [ ]:
# TODO store in unknown_segment


In [ ]:
ipd.Audio(unknown_segment.T, rate=fs)

In [ ]:
detect(unknown_segment, fs)

Looks like our system is pretty robust!

# Final Comments
There are many ways to improve our Shazam system. Many of them have to do with how we compute our spectrogram as well as the various parameters we introduced such as `NEIGHBORHOOD_SIZE`, `AMP_THRESH`, and `HASHES_PER_PEAK`. But, for the most part, this is how Shazam works!

The original Shazam paper uses a different method for matching the fingerprints of audio instead of a simple "most matches => song" scheme, but for our limited database, this works just fine. Check out the original paper if you are curious. If you'd like, you can use the following cells to load your own songs into the database (as long as they are wav files) and try to identify samples of them.

In [ ]:
import csv
def add_to_db(filename):
    fs, audio = wavfile.read(filename)
    hashes = fingerprint(audio, fs)
    with open('database.csv', mode='a') as db_file:
        db_writer = csv.writer(db_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        for hash_pair in hashes:
            db_writer.writerow([hash_pair[0], hash_pair[1], filename])

Run this to add a song to the database.

In [ ]:
my_wav_filepath = # path to any WAV file you want to add to the database
add_to_db(my_wav_filepath)

# References

[1] *An industrial strength audio search algorithm.* [[Link](http://www.ee.columbia.edu/~dpwe/papers/Wang03-shazam.pdf)].  
[2] *Audio fingerprinting with Python and Numpy.* [[Link](https://willdrevo.com/fingerprinting-and-audio-recognition-with-python/)].